# Read Granules

In [ ]:
import starepandas
import netCDF4
import matplotlib.pyplot as plt
import numpy
import pandas
import time
import copy
import geopandas
import pystare

## Manually reading 

In [ ]:
vnp03_name = '../tests/data/granules/VNP03DNB.A2020219.0742.001.2020219124651.nc'
vnp03 = starepandas.io.granules.VNP03DNB(vnp03_name)
vnp03.read_latlon()
vnp03.read_data()
vnp03 = vnp03.to_df(xy=True)

## MOD05

In [ ]:
fname = '../tests/data/granules/MOD05_L2.A2019336.0000.061.2019336211522.hdf'
mod05 = starepandas.read_granule(fname, sidecar=True, latlon=True, xy=True, read_timestamp=True)
sids = mod05.to_array(mod05._sid_column_name)
mod05.to_sidecar('test.nc')

In [ ]:
mod05

In [ ]:
geom = geopandas.points_from_xy(mod05.lon, mod05.lat)
trixels = mod05.make_trixels(n_workers=6)
mod05.set_trixels(trixels, inplace=True)
mod05.set_geometry(geom, inplace=True)
mod05 = mod05.set_crs(4326)

### Subset Loading

In [ ]:
import shapely
#roi = shapely.geometry.Polygon([[-30, 50],[-30, 55],[-25, 55],[-25, 50]])
roi = shapely.geometry.Polygon([[-120, 37],[-120, 38],[-119, 38],[-119, 37]])
roi = geopandas.GeoDataFrame(geometry=[roi], crs=4326)
roi = starepandas.STAREDataFrame(roi)
roi['sids'] = roi.make_sids(resolution=10)
sids = roi.sids.iloc[0]

In [ ]:
roi.plot(trixels=False)

In [ ]:
#fname = '../tests/data/granules/MOD05_L2.A2019336.0000.061.2019336211522.hdf'
fname = '/home/griessbaum/MOD09.A2012244.1835.006.2015248083747.hdf'
mod = starepandas.io.granules.Mod09(fname)
mod.read_sidecar_index()

In [ ]:
intersects = pystare.intersects(sids, mod.sids.flatten())
intersects = numpy.reshape(intersects, mod.sids.shape)
intersects

In [ ]:
import pyhdf
import time
hdf = pyhdf.SD.SD(fname)

start = time.time()
for i in range(0, 100):
    d = hdf.select('Longitude').get()#[intersects]
time.time() - start

## VNP02

In [ ]:
vnp02_name = '../tests/data/granules/VNP02DNB.A2020219.0742.001.2020219125654.nc'
vnp02 = starepandas.read_granule(vnp02_name, sidecar=False, latlon=False)

In [ ]:
vnp02 = vnp03.join(vnp02)

In [ ]:
vnp02[(vnp02.land_water_mask==2) | (vnp02.land_water_mask==4) | (vnp02.land_water_mask==6) | (vnp02.land_water_mask==7)]

## VNP CLDMSK

In [ ]:
cldmsk_name = '../tests/data/granules/CLDMSK_L2_VIIRS_SNPP.A2020219.0742.001.2020219190616.nc'

In [ ]:
cldmsk = starepandas.io.granules.CLDMSKL2VIIRS(cldmsk_name)

In [ ]:
cldmsk.read_latlon()
cldmsk.read_data()
cldmsk.to_df()

## Plotting

In [ ]:
path = '../tests/data/granules/VNP03DNB.A2020219.0742.001.2020219124651.nc'
vnp03dnb = starepandas.io.granules.VIIRSL2(path)
vnp03dnb.read_latlon()

path = '../tests/data/granules/VNP03MOD.A2020219.0742.001.2020219124651.nc'
vnp03mod = starepandas.io.granules.VIIRSL2(path)
vnp03mod.read_latlon()

path = '../tests/data/granules/CLDMSK_L2_VIIRS_SNPP.A2020219.0742.001.2020219190616.nc'
cldmsk = starepandas.io.granules.VIIRSL2(path)
cldmsk.read_latlon()

In [ ]:
fig, ax = plt.subplots(figsize=(5,5), dpi=200)

n = 20
ns = 0
nt = 0

ax.grid('on')

ax.plot(cldmsk.lon[ns:ns+n, nt:nt+n].flatten(), cldmsk.lat[ns:ns+n, nt:nt+n].flatten(), 
        color='r', lw=0, marker='.')

ax.plot(vnp03mod.lon[ns:ns+n, nt:nt+n].flatten(), vnp03mod.lat[ns:ns+n, nt:nt+n].flatten(), 
        color='g',lw=0, marker='.')

ax.plot(vnp03dnb.lon[ns:ns+n, nt:nt+n].flatten(), vnp03dnb.lat[ns:ns+n, nt:nt+n].flatten(), 
        color='b',lw=0, marker='.')